In [52]:
# Activate Conda Env "Ultra" As the Kernel
import torch
import pandas as pd

In [53]:
id2entity, id2relation = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/wn18rr_id2name.pt')

In [61]:
split = 'valid' # the split
ratio = 0.1 # the top r ratio of edges

if split == 'valid':
    data_index = 1
if split == 'test':
    data_index = 2

output_file = f'/storage/ryoji/Graph-Transformer/NBFNet-PyG/explanation/NBFNet/WN18RR/explanation_output/{split}_inspect_hard_edge_mask_top_ratio_{ratio}.pt'

outputs = torch.load(output_file)
output_df = pd.DataFrame(outputs)

In [62]:
display(output_df)

,Ranking,Heads,Tails,Rel,Mode,tail_included_in_ego,head_included_in_expl,tail_included_in_expl,tail_connected_to_head_in_expl,num_nodes_ego,num_edges_ego,num_nodes_expl,num_edges_expl
0,1,1789,9534,5,1,True,True,True,True,14873.0,38370.0,2887.0,3837.0
1,1,35650,23897,1,1,True,True,True,True,14645.0,35003.0,1653.0,3500.0
2,7999,25316,19796,3,1,False,True,False,False,14494.0,34160.0,2800.0,3416.0
3,7,37750,4842,9,1,True,True,True,False,9230.0,18013.0,1631.0,1801.0
4,40300,25541,8811,3,1,False,True,False,False,4847.0,7893.0,545.0,789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063,1,6497,16457,1,0,True,True,True,True,7580.0,15446.0,826.0,1544.0
6064,6,5538,5814,2,0,True,True,True,True,6136.0,13111.0,1112.0,1311.0
6065,1,13787,2914,10,0,True,True,True,True,7628.0,14453.0,789.0,1445.0
6066,9,34493,36131,1,0,True,True,True,True,6768.0,13781.0,830.0,1378.0


In [63]:
print(f"The mean number of nodes in expl: {output_df['num_nodes_expl'].mean()}")
print(f"The mean number of edges in expl: {output_df['num_edges_expl'].mean()}")

The mean number of nodes in expl: 1521.63216875412
The mean number of edges in expl: 2522.6094264996705


In [64]:
# Inclusion rate
inclusion_ego = output_df[output_df['tail_included_in_ego'] == True]
print(f'6-hop ego network inclusion rate: {inclusion_ego.shape[0]/output_df.shape[0]}')

inclusion_expl_tail = output_df[output_df['tail_included_in_expl'] == True]
tail_expl_inclusion_rate = inclusion_expl_tail.shape[0]/output_df.shape[0]
print(f'Tail in Expl inclusion rate: {tail_expl_inclusion_rate}')

inclusion_expl_head = output_df[output_df['head_included_in_expl'] == True]
head_expl_inclusion_rate = inclusion_expl_head.shape[0]/output_df.shape[0]
print(f'Head in Expl inclusion rate: {head_expl_inclusion_rate}')

connected_expl = output_df[output_df['tail_connected_to_head_in_expl'] == True]
connected_rate = connected_expl.shape[0]/output_df.shape[0]
print(f'Tail and Head connected in Expl rate: {connected_rate}')

6-hop ego network inclusion rate: 0.8655240606460118
Tail in Expl inclusion rate: 0.8300922874093606
Head in Expl inclusion rate: 0.9375411997363217
Tail and Head connected in Expl rate: 0.7318721160184575


In [65]:
# Function to calculate hits@N for each mode
def calculate_hits(df, ranks):
    hits_results = {}
    for n in ranks:
        hits_at_n = df[df['Ranking'] <= n].shape[0]
        total = df.shape[0]
        hits_results[f'hits@{n}'] = hits_at_n / total
    return hits_results

In [66]:
# Specify the ranks you want to calculate hits for
ranks = [1, 3, 10]

# Calculate hits for each mode
hits_results = {
    'All Expl': calculate_hits(output_df, ranks),
    'Tail in Expl': calculate_hits(inclusion_expl_tail, ranks),
    'Head in Expl': calculate_hits(inclusion_expl_head, ranks),
    'Tail and Head connected in Expl': calculate_hits(connected_expl, ranks)
}

# Create a DataFrame from the results
hits_df = pd.DataFrame(hits_results).T

# Add inclusion rates to the DataFrame
hits_df['Ratio'] = [1, tail_expl_inclusion_rate, head_expl_inclusion_rate, connected_rate]

# Display the results
display(hits_df)

,hits@1,hits@3,hits@10,Ratio
All Expl,0.478411,0.549110,0.623105,1.000000
Tail in Expl,0.576335,0.661505,0.750447,0.830092
Head in Expl,0.507646,0.581649,0.657585,0.937541
Tail and Head connected in Expl,0.644675,0.735870,0.824589,0.731872
